# Titanic - Machine Learning from Disaster


- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro arquivo dessa análise](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)** (para visualizar a parte 1 completa, visite esse link)

Importando o pandas

In [1]:
import pandas as pd

Visualizando a base de treino

In [2]:
treino = pd.read_csv('train.csv')

Visualizando a base de teste

In [3]:
teste = pd.read_csv('test.csv')

- **Fazendo o mesmo tratamento inicial que fizemos na aula anterior**

Eliminando as colunas com elevada cardinalidade

In [4]:
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)


Usando a média para substituir valores nulos na coluna de idade

In [5]:
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()


Tratando a coluna Embarked da base de treino usando a moda 

In [6]:
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

E também a coluna Fare da base de teste usando a média

In [7]:
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

Verificando as colunas de texto na base de treino

In [8]:
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

Verificando os valores na coluna Sex

In [9]:
treino.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

e na coluna Embarked

In [10]:
treino.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

- Para tratar a coluna Sex, podemos criar uma nova coluna chamada **"Male_Check"** que **vai receber 1 se o gênero for masculino e 0 se o gênero for feminino**

Usando uma lambda function para fazer esse tratamento

In [11]:
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

Verificando os valores

In [12]:
treino[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
dtype: int64

- O próximo passo é fazer o mesmo para a base de teste

Usando uma lambda function para fazer esse tratamento

In [13]:
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

 Verificando os valores

In [14]:
teste[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
dtype: int64

- Agora, para tratar a coluna **Embarked** usando o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá **criar uma nova coluna para cada um dos rótulos da coluna original**

Importando o OneHotEncoder

In [15]:
from sklearn.preprocessing import OneHotEncoder

Criando o encoder

In [16]:
ohe = OneHotEncoder(handle_unknown='ignore')

Fazendo o fit com os dados

In [17]:
ohe = ohe.fit(treino[['Embarked']])

Fazendo a transformação

In [18]:
ohe.transform(treino[['Embarked']]).toarray()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

Transformando esse resultado em um DataFrame

In [19]:
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())
ohe_df.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


Podemos agora adicionar essa coluna na nossa base de treino

In [20]:
treino = pd.concat([treino, ohe_df], axis=1)

Verificando os valores

In [21]:
treino[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0.0         0.0         1.0           646
C         1.0         0.0         0.0           168
Q         0.0         1.0         0.0            77
dtype: int64

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

Transformando esse resultado em um DataFrame

In [22]:
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())

Adicionando o resultado na base de teste

In [23]:
teste = pd.concat([teste, ohe_df], axis=1)

Verificando também os valores

In [24]:
teste[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0.0         0.0         1.0           270
C         1.0         0.0         0.0           102
Q         0.0         1.0         0.0            46
dtype: int64

### Usando essa nova base no modelo

Visualizando a base

In [28]:
treino.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,1,0.0,0.0,1.0
1,2,1,1,38.0,1,0,71.2833,0,1.0,0.0,0.0
2,3,1,3,26.0,0,0,7.9250,0,0.0,0.0,1.0
3,4,1,1,35.0,1,0,53.1000,0,0.0,0.0,1.0
4,5,0,3,35.0,0,0,8.0500,1,0.0,0.0,1.0


Podemos então apagar essas duas colunas que já tratamos

In [29]:
treino = treino.drop(['Sex', 'Embarked'], axis=1)
teste = teste.drop(['Sex', 'Embarked'], axis=1)

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Importando o train_test_split

In [31]:
from sklearn.model_selection import train_test_split

Separando a base de treino em X e y

In [30]:
X = treino.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

Separando em treino e validação

In [32]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

- Para a **árvore de classificação**

Fazendo a importação

In [48]:
from sklearn import tree

Criando o classificador

In [49]:
clf_ac = tree.DecisionTreeClassifier(random_state=425)

Fazendo o fit com os dados

In [50]:
clf_ac = clf_ac.fit(X_train, y_train)

Fazendo a previsão

In [51]:
y_pred_ac = clf_ac.predict(X_val)

- Para o **KNeighborsClassifier**

Importando

In [52]:
from sklearn.neighbors import KNeighborsClassifier

Criando o classificador

In [53]:
clf_knn = KNeighborsClassifier(n_neighbors=3)

Fazendo o fit com os dados

In [54]:
clf_knn = clf_knn.fit(X_train, y_train)

Fazendo a previsão

In [55]:
y_pred_knn = clf_knn.predict(X_val)

- E para a **Regressão Logística**

Importando

In [56]:
from sklearn.linear_model import LogisticRegression

Criando o classificador

In [57]:
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

Fazendo o fit com os dados

In [58]:
clf_rl = clf_rl.fit(X_train, y_train)

Fazendo a previsão

In [59]:
y_pred_rl = clf_rl.predict(X_val)

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

Importando

In [60]:
from sklearn.metrics import accuracy_score

Para a árvore

In [61]:
accuracy_score(y_val, y_pred_ac)

0.7457627118644068

Para o knn

In [62]:
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

Para a regressão logística

In [63]:
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

- Avaliando a **matriz de confusão**

Importando

In [64]:
from sklearn.metrics import confusion_matrix

Para a árvore

In [65]:
confusion_matrix(y_val, y_pred_ac)

array([[137,  38],
       [ 37,  83]], dtype=int64)

Para o knn

In [66]:
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]], dtype=int64)

Para a regressão logística

In [67]:
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

Visualizando o X_train

In [68]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0.0,0.0,1.0
718,3,29.699118,0,0,15.5000,1,0.0,1.0,0.0
685,2,25.000000,1,2,41.5792,1,1.0,0.0,0.0
73,3,26.000000,1,0,14.4542,1,1.0,0.0,0.0
882,3,22.000000,0,0,10.5167,0,0.0,0.0,1.0


Visualizando a base de teste

In [69]:
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0.0,1.0,0.0
1,893,3,47.0,1,0,7.0000,0,0.0,0.0,1.0
2,894,2,62.0,0,0,9.6875,1,0.0,1.0,0.0
3,895,3,27.0,0,0,8.6625,1,0.0,0.0,1.0
4,896,3,22.0,1,1,12.2875,0,0.0,0.0,1.0


Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id

In [70]:
X_teste = teste.drop('PassengerId',axis=1)

Utilizando a regressão logística na base de teste

In [71]:
y_pred = clf_rl.predict(X_teste)

Criando uma nova coluna com a previsão na base de teste

In [72]:
teste['Survived'] = y_pred

Selecionando apenas a coluna de Id e Survived para fazer o envio

In [73]:
base_envio = teste[['PassengerId','Survived']]

Exportando para um csv

In [74]:
base_envio.to_csv('resultados2.csv',index=False)